
# Coding Brick 2, Stat 4770/7770, Fall 2024
 
November 2024



**Introduction**

* In this homework you will download an html file from Yahoo Finance that contains a table of stock prices, trade volumes etc.
* You can look at an example web page by following this link <https://finance.yahoo.com/quote/NVDA/history?p=NVDA>
* Once you have the data in pandas you will clean the file so that it is suitable for analysis.
* Finally, you will create some statistical summaries of the data.

## Q1. 
### 5 pts.

a. Store the stock ticker symbol 'NVDA' in a variable called "stock".

b. Create a new string variable called "url" that is the url of the Yahoo stock web page. It needs to look just like the link above with the ticker symbol embedded (in two places). You shouldn't hard code the stock symbol into the url, but rather use the .format() method on the string together with the {} notation to create the url. An example of this syntax is on the slide "breaking out of a for loop", slide 6.8, Module 2.

c. Print the url variable and make sure it looks exactly like the link above.

In [1]:
# Q1a. Store the stock ticker symbol 'NVDA' in a variable called "stock".
stock = 'NVDA'

# Q1b. Create a new string variable called "url" that is the url of the Yahoo stock web page.
# Use the .format() method to embed the ticker symbol into the URL.
url = "https://finance.yahoo.com/quote/{}/history?p={}".format(stock, stock)

# Q1c. Print the url variable and make sure it looks exactly like the link above.
print(url)

https://finance.yahoo.com/quote/NVDA/history?p=NVDA


## Q2. Read in the html file and list the column names
### 6pts.

> This question uses the Python "requests" library. It is necessary here because Yahoo Finance won't now accept a typical pandas request. Yahoo needs to recognize the user-agent before it is willing send back data. The requests library allows you to rename the user-agent.

a. Use the pd.read_html() command to obtain the data. Recall that the command returns a list of data frames, so pick off the first element. You just have to modify the given line of code, to pick off the first element of the list of data frames. The data frame is called "stock_data". Unlike the class notes, when you read in the html file, do **not** use the 'index_col=0' argument. Without this argument the dates appear in the first column of the data frame, and not as the index. 

b. Obtain and print the dimensions of data frame.

c. Obtain and print the names of all the columns.

d. Obtain and print the data types.

e. What is the problem with the data type in terms of the "Close Close price adjusted for splits." variable, given that we want to do some math on this variable later? 

f. Rename the column called "Close Close price adjusted for splits.", as "Price" and print out the new column names. Make sure that you use the 'inplace=True' argument to the rename method. 

In [5]:
import pandas as pd
import requests
from io import StringIO
print(pd.__version__) # This is your pandas version number which can help in debugging.

# Code to encourage Yahoo to think the request is coming from a human.
# You need to run this!

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://www.google.com",
    "Connection": "keep-alive",
}
# 2a. Read HTML file and extract the first table
response = requests.get(url, headers=headers)
tables = pd.read_html(response.text)
stock_data = tables[0]

# 2b. Obtain and print the dimensions of the data frame
print("Dimensions of the data frame:", stock_data.shape)

# 2c. Obtain and print the names of all columns
print("Column names:", stock_data.columns)

# 2d. Obtain and print the data types of each column
print("Data types:\n", stock_data.dtypes)

# 2e. Explanation of data type issue
# If we want to perform mathematical operations on the "Close Close price adjusted for splits." column,
# it should ideally be of a numeric type. If it’s currently an object type, this could indicate
# non-numeric entries, such as symbols or NaNs, which would require cleaning.

# 2f. Rename the column "Close* Close price adjusted for splits." to "Price" and print new column names
# First, check for the exact column name that matches.
for col in stock_data.columns:
    if "Close" in col:
        stock_data.rename(columns={col: "Price"}, inplace=True)
        break

print("Updated column names:", stock_data.columns)

2.2.2
Dimensions of the data frame: (256, 7)
Column names: Index(['Date', 'Open', 'High', 'Low', 'Close Close price adjusted for splits.',
       'Adj Close Adjusted close price adjusted for splits and dividend and/or capital gain distributions.',
       'Volume'],
      dtype='object')
Data types:
 Date                                                                                                  object
Open                                                                                                  object
High                                                                                                  object
Low                                                                                                   object
Close Close price adjusted for splits.                                                                object
Adj Close Adjusted close price adjusted for splits and dividend and/or capital gain distributions.    object
Volume                                       

/var/folders/d7/v9f8t0990c19nt1jnfk2mbnr0000gn/T/ipykernel_25029/2578690544.py:18: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


## Q3
### 6pts.

* It turns out that there are some row(s) in the data frame that are causing trouble. They have a character string in them and the row(s) is about half way down the data frame. Because of this, the Price column is treated as a string type. 
* You need to turn this column into a **numeric** column.
* pandas has a function to do this and it is called 'to_numeric', and with the 'errors' argument set to 'coerce', it will turn any strings into NaN.  
* You can then remove the row with the NaN value by using the .dropna() method and including the argument 'inplace=True'.

a. Write a for loop that prints out every single entry of the Price column (this will help you see the underlying problem).

b. Use the 'pd.to_numeric' function to turn the stock_data Price column to a numeric type. Make sure you assign this modified version of the Price column, back into the data frame. 

c. Print the data types in the data frame to confirm that this has worked. Price should now be of type 'float64'.

d. Finally remove the row with the NaN in it, using .dropna. Print the dimensions of the modified data frame.

In [6]:
# Q3 code goes here

#3a. (2pts.)
print("Entries in the 'Price' column:")
for entry in stock_data['Price']:
    print(entry)
#3b. (2pts.)
stock_data['Price'] = pd.to_numeric(stock_data['Price'], errors='coerce')

#3c. (1pt.)
print("\nData types after conversion:")
print(stock_data.dtypes)
#3d. (1pt.)
stock_data.dropna(subset=['Price'], inplace=True)
print("\nDimensions of the data frame after dropping NaN rows:", stock_data.shape)

Entries in the 'Price' column:
144.95
147.63
148.88
145.61
139.91
136.05
135.40
132.76
139.34
141.25
140.52
141.54
140.41
139.56
143.59
143.71
138.00
136.93
135.72
131.60
138.07
134.80
134.81
132.65
132.89
127.72
124.92
122.85
118.85
117.00
121.44
121.40
124.04
123.51
120.87
116.26
116.00
117.87
113.37
115.59
116.78
119.10
0.01 Dividend
119.14
116.91
108.10
106.47
102.83
107.21
106.21
108.00
119.37
117.59
125.61
128.30
126.46
129.37
123.74
128.50
127.25
130.00
124.58
122.86
118.08
116.14
109.02
104.75
104.97
98.91
104.25
100.45
107.27
109.21
117.02
103.73
111.59
113.06
112.28
114.25
122.59
123.54
117.93
121.09
117.99
126.36
128.44
129.24
127.40
134.91
131.38
128.20
125.83
128.28
122.67
124.30
123.54
123.99
126.40
126.09
118.11
126.57
130.78
135.58
130.98
131.88
129.61
125.20
0.01 Dividend
120.91
10:1 Stock Splits
121.79
120.89
121.00
122.44
116.44
115.00
109.63
110.50
114.82
113.90
106.47
103.80
94.95
95.39
94.78
92.48
94.36
94.63
91.36
90.40
89.88
88.75
90.41
90.55
92.14
88.79
85.82
8

## Q4
### 9 pts.

Using the Price column in the cleaned data frame find and print:

a. The average price:  .mean()

b. The median price: .median()

c. The standard deviation of the price: .std()

d. The 20th and 80th percentiles of price (put the appropriate cut-off percentages in a list): .quantile([ , ])

> For parts e and f, use the result of .idmax() and .idmin() as an index into the Date column.

e. The day on which the maximum price occurred: .idxmax()

f. The day on which the minimum price occurred: .idxmin()

g. Use the .describe() function for an overall summary of the Price column.



In [7]:
# Q4 code goes here

#4a. (1pt.)
average_price = stock_data['Price'].mean()
print("Average Price:", average_price)
#4b. (1pt.)
median_price = stock_data['Price'].median()
print("Median Price:", median_price)

#4c. (1pt.)
std_price = stock_data['Price'].std()
print("Standard Deviation of Price:", std_price)
#4d. (1pt.)
percentiles = stock_data['Price'].quantile([0.2, 0.8])
print("20th Percentile of Price:", percentiles[0.2])
print("80th Percentile of Price:", percentiles[0.8])
#4e. (2pts.)
max_price_date = stock_data.loc[stock_data['Price'].idxmax(), 'Date']
print("Date of Maximum Price:", max_price_date)

#4f. (2pts.)
min_price_date = stock_data.loc[stock_data['Price'].idxmin(), 'Date']
print("Date of Minimum Price:", min_price_date)
#4g. (1pt.)
price_summary = stock_data['Price'].describe()
print("\nSummary of Price column:\n", price_summary)

Average Price: 95.75219123505975
Median Price: 94.29
Standard Deviation of Price: 30.215619013670445
20th Percentile of Price: 61.53
80th Percentile of Price: 125.2
Date of Maximum Price: Nov 7, 2024
Date of Minimum Price: Dec 6, 2023

Summary of Price column:
 count    251.000000
mean      95.752191
std       30.215619
min       45.500000
25%       72.130000
50%       94.290000
75%      122.630000
max      148.880000
Name: Price, dtype: float64


## Q5.
###  6 pts.
Note that the first column in the data frame contains date data but it is being viewed as type 'string'.

a. Overwrite and cast this column as a date column using the pd.to_datetime() function, as in the Module 4 notes, slide 6.6.

b. Print the type of the first element in the new Date column to confirm it is now a valid date type. So long as you see ".Timestamp" as part of the type, you have this correct. 

In [8]:
# Q5 code goes here

#5a. (3pts.)
stock_data['Date'] = pd.to_datetime(stock_data['Date'])

# 5b. (3pts.)
first_date_type = type(stock_data['Date'].iloc[0])
print("Type of first element in the Date column:", first_date_type)

Type of first element in the Date column: <class 'pandas._libs.tslibs.timestamps.Timestamp'>


## Q6.
### 8 pts.

a. Create a new variable in the data frame called 'DayOfWeek' by using the .dt.day_name() method.

b. Create a new variable in the data frame called 'MonthOfYear' by using the .dt.month_name() method.

c. Print out all the columns in the stock_data data frame to confirm that the new columns are there. 

In [9]:
# Q6 code goes here

#6a. Create day of week. (3pts.)
stock_data['DayOfWeek'] = stock_data['Date'].dt.day_name()

#6b. Create month of year (3pts.)
stock_data['MonthOfYear'] = stock_data['Date'].dt.month_name()

#6c. (2pt.)
print("Columns in stock_data DataFrame:", stock_data.columns)
print(stock_data.head())  # Display the first few rows to confirm the new columns are populated correctly

Columns in stock_data DataFrame: Index(['Date', 'Open', 'High', 'Low', 'Price',
       'Adj Close Adjusted close price adjusted for splits and dividend and/or capital gain distributions.',
       'Volume', 'DayOfWeek', 'MonthOfYear'],
      dtype='object')
        Date    Open    High     Low   Price  \
0 2024-11-11  148.68  148.77  143.57  144.95   
1 2024-11-08  148.77  149.77  146.26  147.63   
2 2024-11-07  146.39  148.93  146.17  148.88   
3 2024-11-06  142.96  146.49  141.96  145.61   
4 2024-11-05  137.45  140.37  137.33  139.91   

  Adj Close Adjusted close price adjusted for splits and dividend and/or capital gain distributions.  \
0                                             144.95                                                   
1                                             147.63                                                   
2                                             148.88                                                   
3                                     

## Q7.
### 5 pts.

a. Print out the number of observations for each day of the week. Use the .value_counts() method.

a. Print out the number of observations for each month of the year. Use the .value_counts() method.

In [10]:
# Q7 code goes here

#Q7a. (3pts.)
day_of_week_counts = stock_data['DayOfWeek'].value_counts()
print("Number of observations for each day of the week:")
print(day_of_week_counts)
#Q7b. (2pts.)
month_of_year_counts = stock_data['MonthOfYear'].value_counts()
print("\nNumber of observations for each month of the year:")
print(month_of_year_counts)

Number of observations for each day of the week:
DayOfWeek
Tuesday      52
Friday       51
Wednesday    51
Thursday     50
Monday       47
Name: count, dtype: int64

Number of observations for each month of the year:
MonthOfYear
October      23
August       22
July         22
May          22
April        22
January      21
November     20
September    20
March        20
February     20
December     20
June         19
Name: count, dtype: int64


## Q8
### 5pt.

a. It is common to want to transform prices into returns. The formula is $R_t = \frac{P_t - P_{t-1}}{P_{t-1}}$. Here, $R_t$ is the return for time t, and $P_t$ is the price at time t. Fortunately pandas has a built in method to do this which is called '.pct_change()'. Use this function to add a new column to the existing data frame called 'Return'. There is a subtelty here in that the dataframe is ordered from most recent price in the first row, to oldest price in the last row, which is unusual for a time series. For the pct_change method to work properly we need to first reverse the row order in the data frame.

A quick way to reverse the data frame is provided at the top of the code block below.

Print the first 5 rows of the reordered data frame to confirm that it now starts in September, and that the Return has been calculated as expected. The first element in the Return column should be an NaN, as it has no preceding price.

b. Using the .groupby method, find and print the average return for each *day of the week*.

c. Using the .groupby method, find and print the average return for each *month of the year*.

In [11]:
# Q8 code goes here

# Reverse the order of the rows in the data frame.
stock_data = stock_data[::-1].copy()

#8a. (3pts.)
stock_data['Return'] = stock_data['Price'].pct_change()

# Print the first 5 rows to confirm the reversal and the calculation of Return
print("First 5 rows of the data frame with Returns calculated:")
print(stock_data.head())

#8b. (1pt.)
average_return_by_day = stock_data.groupby('DayOfWeek')['Return'].mean()
print("\nAverage Return for each day of the week:")
print(average_return_by_day)

#8c. (1pts)
average_return_by_month = stock_data.groupby('MonthOfYear')['Return'].mean()
print("\nAverage Return for each month of the year:")
print(average_return_by_month)

First 5 rows of the data frame with Returns calculated:
          Date   Open   High    Low  Price  \
255 2023-11-13  48.32  49.12  48.10  48.62   
254 2023-11-14  49.68  49.83  49.04  49.66   
253 2023-11-15  49.94  49.96  48.20  48.89   
252 2023-11-16  48.68  49.53  48.33  49.48   
251 2023-11-17  49.52  49.72  49.01  49.30   

    Adj Close Adjusted close price adjusted for splits and dividend and/or capital gain distributions.  \
255                                              48.61                                                   
254                                              49.64                                                   
253                                              48.87                                                   
252                                              49.47                                                   
251                                              49.28                                                   

        Volume  DayOfWeek Mont